In [1]:
from configuracoes_notebooks import set_proj_dir
set_proj_dir()

O diretorio do seu projeto é coleta_cebrap
Caminho absoluto do diretorio encontrado C:\Users\x526378\Desktop\projetos\cebrap\coleta_cebrap
Caminho no path.


In [2]:
import os
import geopandas as gpd

from notebooks.jupyter import utils
from utils import (
    get_data_diretorio,
    save_parquet_excel
)

# Cobertura Vegetal por habitante (m²/hab)

In [3]:
data_path = get_data_diretorio()

# Dependências

Este notebook é dependente do asset resultante do notebook '[cebrap]cobertura_vegetal_por_distrito'

In [4]:
dist_cobveg_path = os.path.join(
    data_path,
    'assets',
    'cobertura_vegetal', 
    'distritos_cobertura_vegetal.parquet'
)
gdf_dist_cobveg = gpd.read_parquet(
    dist_cobveg_path
)

?? -> o resultado de gdf['geometry'].area é m² mesmo, né?

# Cálculo per capita (m²/hab)

In [5]:
gdf_dist_cobveg.sample(2)

,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,area_dist_cobveg,area_distrito,pct_dist_cobveg
67,3550308,São Paulo,355030868,SACOMA,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,14.610797,261436,113894,"POLYGON ((338605.158 7385601.06, 338598.678 73...",2.575763e+06,1.460870e+07,0.176317
26,3550308,São Paulo,355030827,CURSINO,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,12.060052,103171,46155,"POLYGON ((335822.146 7385759.521, 335833.043 7...",5.062376e+06,1.205855e+07,0.419816


In [6]:
gdf_dist_cobveg['cob_veg_per_capita'] = (
    gdf_dist_cobveg['area_dist_cobveg']/gdf_dist_cobveg['total_pop']
)

In [7]:
gdf_dist_cobveg.sample(2)

,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,area_dist_cobveg,area_distrito,pct_dist_cobveg,cob_veg_per_capita
17,3550308,São Paulo,355030818,CANGAIBA,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,13.757295,141172,58654,"POLYGON ((346119.127 7399284.114, 346111.907 7...",4.631445e+06,1.375446e+07,0.336723,32.807110
38,3550308,São Paulo,355030839,JACANA,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,7.473932,87329,34705,"POLYGON ((339057.719 7403206.175, 339043.253 7...",1.542754e+06,7.472732e+06,0.206451,17.666003


In [8]:
sum(gdf_dist_cobveg['cob_veg_per_capita'].isnull())

0

Como precisamos saber qual a unidade de medida, vamos confirmar as informações do crs:

In [9]:
gdf_dist_cobveg.crs

<Projected CRS: EPSG:31983>
Name: SIRGAS 2000 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Brazil - between 48°W and 42°W, northern and southern hemispheres, onshore and offshore.
- bounds: (-48.0, -33.5, -42.0, 5.13)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

Segundo as informações da projeção, a unidade é em metros e não há a necessidade de alterar.

In [10]:
max_pct=gdf_dist_cobveg['cob_veg_per_capita'].max()
max_name=gdf_dist_cobveg['NM_DIST'].loc[gdf_dist_cobveg['cob_veg_per_capita']==max_pct].iloc[0]
print(f'{max_name} é o distrito com a maior área de cobertura vegetal per capita: {round (max_pct, 2)} m²/hab')

MARSILAC é o distrito com a maior área de cobertura vegetal per capita: 17598.75 m²/hab


In [11]:
min_pct=gdf_dist_cobveg['cob_veg_per_capita'].min()
min_name = gdf_dist_cobveg['NM_DIST'].loc[gdf_dist_cobveg['cob_veg_per_capita']==min_pct].iloc[0]
print(f'{min_name} é o distrito com a menor porcentagem de cobertura vegetal: {round(min_pct, 2)} m²/hab')

REPUBLICA é o distrito com a menor porcentagem de cobertura vegetal: 4.61 m²/hab


# Salvar gdf

In [12]:
save_parquet_excel(
    gdf_dist_cobveg, 
    fname= 'cobertura_vegetal_per_capita',
    data_path = data_path,
    data_subpath = os.path.join(
        'assets',
        'cobertura_vegetal'
    )
)